# VIX ETFによるアーブ

In [ ]:
from quantopian.interactive.data.quandl import cboe_vix
from odo import odo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def get_vix():
    df_vix = odo(cboe_vix, pd.DataFrame)
    df_vix['asof_date'] = pd.to_datetime(df_vix['asof_date'])
    df_vix = df_vix.set_index(['asof_date'])
    df_vix.index.names = ['Date']
    df_vix.index = df_vix.index.tz_localize('UTC')
    df_vix = df_vix.sort_index()
    df_vix = df_vix.rename(columns={'vix_open':'open_price', 'vix_high':'high', 'vix_low':'low','vix_close':'close_price',})
    df_vix['price'] = df_vix['close_price']
    return df_vix

start = "2013-1-1"
end = "2017-8-11"

XIV = symbols('XIV')
TVIX = symbols('TVIX')
VXX = symbols('VXX')
SPY = symbols('SPY')
#
p = get_pricing([XIV, TVIX, VXX, SPY], start_date=start, end_date=end, frequency='daily', fields=['open_price', 'high', 'low', 'close_price', 'volume', 'price',])
df_vix = get_vix()

## データ取得確認

In [ ]:
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1,3,1)
ax.plot(p['close_price'][SPY])
ax = fig.add_subplot(1,3,2)
ax.plot(df_vix['close_price'])
ax = fig.add_subplot(1,3,3)
ax.scatter(p['close_price'][XIV].pct_change(), p['close_price'][TVIX].pct_change())

## TVIXとXIVとのアービトラージ機会の確認

In [ ]:
xiv_c = p['close_price'][XIV].pct_change()
tvix_c = p['close_price'][TVIX].pct_change()
ln_xiv_c = xiv_c.apply(lambda x: np.log(x+1.0))
ln_tvix_c = tvix_c.apply(lambda x: np.log(x+1.0))
vix = df_vix.ix[ln_tvix_c.index]['close_price']
ln_vix = vix.apply(np.log)
diff = -ln_xiv_c-ln_tvix_c*0.5
plt.scatter(xiv_c, diff, c=ln_vix, cmap=cm.Reds )
plt.colorbar()

- XIVが1日でだいたい5%以上動くようなときには、XIVのショートとTVIXのショート（元本はXIVの半分）の組み合わせで収益がでそうということが確認できた

In [ ]:
diff.hist(bins=50, alpha=0.3)
(-ln_tvix_c).hist(bins=50, alpha=0.3)
ln_xiv_c.hist(bins=50, alpha=0.3)

- テイルが小さくなり、収益性が安定することが確認できる

## VIXレベルとの関係

In [ ]:
plt.scatter(vix, diff, c=ln_vix, cmap=cm.Reds)
plt.colorbar()

- VIXが17あたりからうえではかなり確実性高く大きく儲かるが、それ以下では損のケースが増える

## VIXの推移と比較して、時期的な塊がないか、局面による違いがないかチェックする

In [ ]:
plt.plot(vix.index, vix, zorder=1, alpha=0.2)
plt.scatter(vix.index, vix, c=diff, s=8, cmap=cm.bwr)
plt.clim(-0.005, 0.005)
plt.colorbar()

- VIXの下降局面で損がでやすい印象がある

## VIXの変化率との関係をチェックする

In [ ]:
fig = plt.figure(figsize=(12, 8))
for i, n in enumerate([1, 2, 5, 10, 20]):
    ax = fig.add_subplot(2,5,i+1)
    ax.set_ylim((-0.025, 0.15))
    ax.scatter(vix.pct_change(n).shift(1), diff, alpha=0.2)　# 当日を除く
ax = fig.add_subplot(2,1,2)
ax.set_ylim((-0.025, 0.05))
ax.scatter(vix.pct_change(1), diff, alpha=0.3) # 当日

- 前日比5%以上上がっているときはほとんど益が出るが、-1%から-10%あたりは損が出やすい。
- 比較期間は当日１日だけが良さそうだが、これはXIVやTVIXの価格連動が一日単位なのと整合的。
- 前日のVIXとは弱い逆相関が見られる。
- VIXの上昇局面に強い傾向も確認できた。

## 保有期間１日と日中だけの違い

In [ ]:
Ln_intraday_xiv_c = (p['close_price'][XIV]/p['open_price'][XIV]).apply(np.log)
Ln_intraday_tvix_c = (p['close_price'][TVIX]/p['open_price'][TVIX]).apply(np.log)
intraday_diff = -Ln_intraday_xiv_c-Ln_intraday_tvix_c*0.5
#plt.scatter(Ln_intraday_xiv_c, , c=ln_vix, cmap=cm.Reds )
#plt.colorbar()
print diff.mean(), intraday_diff.mean()
diff.hist(bins=100, range=(-0.02, 0.05), color='blue', alpha=0.3)
intraday_diff.hist(bins=100, range=(-0.02, 0.05), color='red', alpha=0.3)


- 平均では１日より日中だけのほうが収益率は低いが、極端な収益が出る日を除くと、日中だけのほうが収益の確実性は高い。
- 保有時間の短さ、夜間の情勢の見極めができる点では、日中だけで取引するのが良いと思われる。
- ただし、日中だけの取引の場合、組み合わせポジションが約定できないリスクがある。

## XIVとTVIXの比率とパフォーマンスの偏差の確認

In [ ]:
std, intraday_std = list(), list()
for m in range(40, 75):
    d = (-ln_xiv_c-ln_tvix_c*m/100.0)
    d_intraday = (-Ln_intraday_xiv_c-Ln_intraday_tvix_c*m/100.0)
    std.append(d.mean()/d.std())
    intraday_std.append(d_intraday.mean()/d_intraday.std())
plt.plot(range(40, 75), std, color='blue')
plt.plot(range(40, 75), intraday_std, color='red')

- 日中だけのほう１日よりも偏差は小さく、安定的になることが確認できた。
- 組み合わせ比率は0.5ではなく、0.55くらいか。ただし、ここでは対数の足し算で単純に比率をかけているだけであることに注意。

## ボリュームとの関係の確認

In [ ]:
ln_turnover_vxx= (p['close_price'][VXX]*p['volume'][VXX]).apply(np.log)
ln_turnover_xiv= (p['close_price'][XIV]*p['volume'][XIV]).apply(np.log)
ln_turnover_tvix= (p['close_price'][TVIX]*p['volume'][TVIX]).apply(np.log)
fig = plt.figure(figsize=(12, 4))
for i, t in enumerate([ln_turnover_vxx, ln_turnover_xiv, ln_turnover_tvix]):
    ax = fig.add_subplot(3,4,i+1)
    ax.set_ylim((-0.025, 0.15))
    ax.scatter(t.shift(1), diff, alpha=0.3) # 前日
    ax = fig.add_subplot(3,4,i+5)
    ax.set_ylim((-0.025, 0.15))
    ax.scatter(t, diff, alpha=0.3)
    ax = fig.add_subplot(3,4,i+9)
    ax.set_ylim((-0.025, 0.05))
    ax.scatter(t.pct_change(10).shift(1), diff, alpha=0.3) # 前10日から前日にかけての変化
ax = fig.add_subplot(3,4,4)
ax.set_ylim((-0.025, 0.15))
plt.scatter((ln_turnover_vxx-ln_turnover_xiv).shift(1), diff, alpha=0.3) # 前日
ax = fig.add_subplot(3,4,8)
ax.set_ylim((-0.025, 0.15))
plt.scatter(ln_turnover_vxx-ln_turnover_xiv, diff, alpha=0.3)
ax = fig.add_subplot(3,4,12)
ax.set_ylim((-0.025, 0.05))
ax.set_xlim((-5, 5))
plt.scatter((ln_turnover_vxx-ln_turnover_xiv).pct_change(10).shift(1), diff, alpha=0.3) # 前10日から前日にかけての変化


- 取引金額が大きい時のほうが収益が出る傾向がある。
- 前日の取引金額はだいぶん相関が弱くなる
- 前々日から前日への変化は使えないが、期間を長めに取ると少し傾向が出る

# TODO

- コンタンゴとの関係
- 先物の出来高、特にVX1とVX2の比率とか残存期間とかとの関係
- VX1の予測、VX2の予測でコンタンゴの動きを予測して比較
- ここまでは当日の諸々の関係しか見ていないので、全日夜間までの動きとの関係を調べる
- HMMでやってみる

In [ ]:
plt.scatter(diff.shift(1),diff,alpha=0.3)

In [ ]:
plt.scatter(diff.rolling(window=5).mean().shift(1),diff,alpha=0.3)

In [ ]:
plt.scatter(diff.apply(abs).rolling(window=3).mean().shift(1),diff,alpha=0.1)

In [ ]:
plt.scatter(p['close_price'][SPY].rolling(window=5).std().shift(1),diff,alpha=0.3)

In [ ]:
etfs = [ 'LABD', ] #'VXX', 'XIV',
start = "2014-1-1"
end = "2017-6-1"

df = get_pricing(etfs, start_date=start, end_date=end, frequency='minute', fields='price')
df = df.rename(columns=dict([(s, s.symbol) for s in df.columns]))
df.index =df.index.tz_convert("US/Eastern")
df = df.fillna(method="ffill", axis=1)

In [ ]:
def get_at_time(df, attime, dates=None):
    x = df.at_time(attime)
    x.index = x.index.date
    if dates:
        return x.ix[dates]
    return x

df_1600 = get_at_time(df, "16:00")
df_0931 = get_at_time(df, "09:31", df_1600.index.tolist())

array_0931 = get_at_time(df, "09:31", df_1600.index.tolist()).values
array_1200 = get_at_time(df, "12:00", df_1600.index.tolist()).values
array_1500 = get_at_time(df, "15:00", df_1600.index.tolist()).values
array_1545 = get_at_time(df, "15:45", df_1600.index.tolist()).values
array_1600 = df_1600.values
array_prev_1600 = df_1600.shift(1).values

In [ ]:
array_1545_1600 = array_1600 / array_1545 - 1

array_prevclose_0931 = array_0931 / array_prev_1600 - 1
array_prevclose_1200 = array_1200 / array_prev_1600 - 1
array_prevclose_1545 = array_1545 / array_prev_1600 - 1


In [ ]:
import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(20,10))

ax = fig.add_subplot(1,3,1)

ax.scatter(array_prevclose_0931, 
          array_1545_1600, 
           alpha=0.3)
ax.set_xlabel("prev close to 09:31")
ax.set_ylabel("15:45 to 16:00")

ax = fig.add_subplot(1,3,2)
ax.scatter(array_prevclose_1200, 
          array_1545_1600, 
           alpha=0.3)
ax.set_xlabel("prev close to 12:00")
ax.set_ylabel("15:45 to 16:00")

ax = fig.add_subplot(1,3,3)
ax.scatter(array_prevclose_1545, 
          array_1545_1600, 
           alpha=0.3)
ax.set_xlabel("prev close to 15:45")



In [ ]:
symbols('UPRO').ass